# Wprowadzenie do Spark SQL

## DataFrame w Spark SQL

DataFrame to abstrakcyjna struktura danych w Spark SQL, która reprezentuje tabelę danych z kolumnami i wierszami.
DataFrame jest rozszerzeniem RDD, które jest zbiorem obiektów RDD, w którym każdy obiekt jest strukturą danych zawierającą nazwane kolumny (schemat danych).

DataFrame można tworzyć na różne sposoby:
- z obiektu RDD
- z pliku CSV, JSON, Parquet
- z bazy danych
- z innych źródeł danych
- z innych DataFrame
- z kolekcji danych
- z danych strumieniowych
- z danych zewnętrznych
- z danych z API
 

## Inicjowanie Sparka
Pracę ze Sparkiem zaczynamy od zainicjowania sesji Sparka.
Aby zainicjować Sparka, musimy zaimportować pakiet `findspark` i uruchomić metodę `init()`:

In [ ]:
import findspark
findspark.init() 

Następnie tworzymy obiekt sesji Sparka. Zwróć uwagę na ustawienie nazwy aplikacji Sparka: 

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
# obiekt sesji zwykle ma nazwę "spark"
spark = SparkSession.builder.appName("DataScience").getOrCreate() 
spark

## Tworzenie obiektów DataFrame

W pierwszym ćwiczeniu spróbujemy stworzyć obiekty DataFrame z kolekcji danych, obiektu RDD, pliku CSV oraz pliku Parquet.


### Utworzenie obiektu DataFrame z obiektu RDD

In [ ]:
rdd = spark.sparkContext.parallelize([1, 2, 3, 4, 5])
# musimy zmienić obiekt RDD na obiekt RDD zawierający krotki, ponieważ obiekt RDD musi zawierać strukturę danych
# nie może zawierać pojedynczych wartości
rdd = rdd.map(lambda x: (x, ))
rdd.collect()

Mając obiekt RDD, możemy zmienić go na obiekt DataFrame za pomocą metody `.toDF()`.

In [ ]:
df = rdd.toDF()
df

In [ ]:
# funkcja show() wyświetla zawartość obiektu DataFrame
df.show()

In [ ]:
# funkcja printSchema() wyświetla schemat danych obiektu DataFrame
df.printSchema()

In [ ]:
# funkcja describe() wyświetla statystyki opisowe obiektu DataFrame
df.describe().show()

Jak widać powyżej, obiekt DataFrame zawiera kolumnę z nazwą `_1`. Musimy zmienić nazwę kolumny na bardziej opisową.

In [ ]:
df = rdd.toDF(["value"])
df.show()

Typ danych w obiekcie DataFrame jest automatycznie ustawiany na podstawie danych wejściowych. Możemy sprawdzić typ danych za pomocą metody `dtypes`.

In [ ]:
df.dtypes

Jak widać powyżej, typ danych w kolumnie `value` został ustawiony na `int`. Możemy zobaczyć schemat danych za pomocą metody `schema`.

In [ ]:
df.schema

In [ ]:
df.schema.names


In [ ]:
df.schema.fields

### Utworzenie obiektu DataFrame z kolekcji danych

Użyjemy metody `createDataFrame()` do utworzenia obiektu DataFrame z kolekcji danych.
Wartości w kolumnach obiektu DataFrame mogą być różnych typów danych.

In [ ]:
data = [("John", 25), ("Anna", 23), ("Mike", 30), ("Jane", 22)]

In [ ]:
df = spark.createDataFrame(data, ["name", "age"])
df

In [ ]:
df.show()

### Utworzenie obiektu DataFrame z pliku CSV

DataFrame można również utworzyć z pliku CSV. W tym celu używamy metody `read.csv()`.


In [ ]:
df = spark.read.csv("../../data/stock/stock.csv", header=True, inferSchema=True)
df

In [ ]:
df.show(5)

### Utworzenie obiektu DataFrame z pliku Parquet

Parquet to format pliku binarnego używany w Sparku do przechowywania danych.

In [ ]:
df = spark.read.parquet("../../data/sklep/categories")
df

## Podstawowe akcje na obiektach DataFrame
Akcja to operacja, która zwraca wartość do sterownika programu po przetworzeniu danych.

In [ ]:
df = spark.read.parquet("../../data/sklep/categories")

### show()

`show()` wyświetla zawartość obiektu DataFrame.

In [ ]:
df.show(5)

In [ ]:
df.show(5, truncate=False)  # truncate=False wyświetla całą zawartość kolumny

### printSchema()

`printSchema()` wyświetla schemat danych obiektu DataFrame.

In [ ]:
df.printSchema()    # wyświetla schemat danych obiektu DataFrame

### describe()

`describe()` wyświetla statystyki opisowe obiektu DataFrame.

In [ ]:
df.describe().show()

In [ ]:
df.describe("category_id").show()

In [ ]:
df.describe("category_id", "category_name").show()

### count()

`count()` zwraca liczbę wierszy obiektu DataFrame.

In [ ]:
df.count()

### select()

`select()` zwraca nowy obiekt DataFrame z wybranymi kolumnami.

In [ ]:
df.select("*").show()

In [ ]:
df.select("category_id", "category_name").show(5)       # wybiera kolumny category_id i category

In [ ]:
df.select(df["category_id"], df["category_name"]).show(5)       # można odwoływać się do kolumn za pomocą nawiasów kwadratowych

In [ ]:
df.select(df.category_id, df.category_name).show(5)     # można odwoływać się do kolumn za pomocą kropki

### filter()

`filter()` zwraca nowy obiekt DataFrame z wierszami spełniającymi warunek.


In [ ]:
df2 = df.filter(df["category_id"] == 1)
df2.show()

In [ ]:
df.filter(df["category_id"] == 1).select("category_id", "category_name").show()

In [ ]:
df.filter(df["category_id"] == 1).filter(df["category_name"] == "Football").show()

### distinct()

`distinct()` zwraca obiekt DataFrame bez duplikatów.


In [ ]:
df.select("category_department_id").distinct().show()

### orderBy()

`orderBy()` sortuje wiersze obiektu DataFrame według kolumny.

In [ ]:
df.orderBy("category_id").show(5)

In [ ]:
df.orderBy(df["category_id"].desc()).show(5)        # sortowanie malejące według kolumny category_id

In [ ]:
df.orderBy(df["category_id"].asc(), df["category_name"].desc()).show(5)  # sortowanie rosnące według kolumny category_id oraz malejące według kolumny category_name

### groupBy()

`groupBy()` grupuje wiersze obiektu DataFrame według kolumny.

In [ ]:
df.groupBy("category_id").count().show(5)    # grupuje wiersze według kolumny category_id i zlicza liczbę wierszy w każdej grupie

In [ ]:
df.groupBy("category_id").count().show(5)

In [ ]:
df.groupBy("category_id").agg({"category_name": "count"}).show(5)    # grupuje wiersze według kolumny category_id i zlicza liczbę wierszy w każdej grupie

In [ ]:
df.groupBy("category_id").agg({"category_name": "count", "category_id": "sum"}).show(5)    # grupuje wiersze według kolumny category_id i zlicza liczbę wierszy w każdej grupie

In [ ]:
df.groupBy("category_id").agg({"category_name": "count", "category_id": "sum"}).orderBy("category_id").show(5)    # grupuje wiersze według kolumny category_id i zlicza liczbę wierszy w każdej grupie

### join()

`join()` łączy dwa obiekty DataFrame na podstawie kolumny.

Wykorzystamy dwa obiekty DataFrame: `categories` oraz `products`.


In [ ]:
categories = spark.read.parquet("../../data/sklep/categories")
products = spark.read.parquet("../../data/sklep/products")


In [ ]:
categories.show(5)

In [ ]:
products.show(5)

In [ ]:
categories.join(products, categories["category_id"] == products["product_category_id"]).show(5)

Wyróżniamy kilka typów łączenia obiektów DataFrame:

- inner join - zwraca wiersze, które mają pasujące wartości w obu obiektach DataFrame
- outer join - zwraca wiersze, które mają pasujące wartości w jednym z obiektów DataFrame
- left join - zwraca wiersze, które mają pasujące wartości w lewym obiekcie DataFrame
- right join - zwraca wiersze, które mają pasujące wartości w prawym obiekcie DataFrame
- left semi join - zwraca wiersze z lewego obiektu DataFrame, które mają pasujące wartości w prawym obiekcie DataFrame
- left anti join - zwraca wiersze z lewego obiektu DataFrame, które nie mają pasujących wartości w prawym obiekcie DataFrame
- cross join - zwraca iloczyn kartezjański obu obiektów DataFrame
- natural join - zwraca wiersze, które mają pasujące wartości w obu obiektach DataFrame, ale bez powtarzających się kolumn

 
join_types = ["inner", "cross", "outer", "full", "left_outer", "right_outer"]


In [ ]:
categories.join(products, categories["category_id"] == products["product_category_id"], "inner").show(5)

In [ ]:
categories.join(products, categories["category_id"] == products["product_category_id"], "outer").show(5)

### union()

`union()` łączy dwa obiekty DataFrame w jeden obiekt DataFrame.


In [ ]:
df1 = spark.createDataFrame([(1, "John"), (2, "Anna")], ["id", "name"])
df2 = spark.createDataFrame([(3, "Mike"), (4, "Jane")], ["id", "name"])
df1.show()
df2.show()

In [ ]:
df1.union(df2).show()

### withColumn()

`withColumn()` dodaje nową kolumnę do obiektu DataFrame.


In [ ]:
df.withColumn("new_column", df["category_id"] + 1).show(5)

In [ ]:
df.withColumn("new_column", df["category_id"] + 1).select("category_id", "new_column").show(5)

### drop()

`drop()` usuwa kolumnę z obiektu DataFrame.

In [ ]:
df.drop("category_id").show(5)

### na()

`na()` zwraca obiekt DataFrame z wartościami brakującymi zastąpionymi wartościami.




In [ ]:
# przykładowy obiekt DataFrame z wartościami brakującymi
df = spark.createDataFrame([(1, "John"), (2, None), (3, "Mike"), (4, "Jane")], ["id", "name"])
df.show()


In [ ]:
df.na.fill("Anna").show()   # zastępuje wartości brakujące wartością "Anna"

In [ ]:
df.na.fill({"name": "Anna"}).show()   # zastępuje wartości brakujące wartością "Anna" w kolumnie name

In [ ]:
df.na.drop().show()   # usuwa wiersze z wartościami brakującymi

### replace()

`replace()` zastępuje wartości w obiekcie DataFrame.
Zamiana wartości w obiekcie DataFrame jest możliwa tylko dla kolumn typu string.


In [ ]:
df.replace("John", "Anna").show(5)   # zastępuje wartość "John" wartością "Anna"

### cache()

`cache()` zapisuje obiekt DataFrame w pamięci podręcznej.

In [ ]:
df = spark.createDataFrame([(1, "John"), (2, "Anna"), (3, "Mike"), (4, "Jane")], ["id", "name"])
df.cache()

In [ ]:
df.show()

### unpersist()

`unpersist()` usuwa obiekt DataFrame z pamięci podręcznej. Pozwala to na zwolnienie pamięci podręcznej.

In [ ]:
df.unpersist()

In [ ]:
df.show(5)

In [ ]:
df.take(5)

### collect(), take(), head(), first(), show(), toPandas()

`collect()` zwraca wszystkie wiersze obiektu DataFrame do sterownika programu.
`take(n)` zwraca n pierwszych wierszy obiektu DataFrame do sterownika programu.
`head(n)` zwraca n pierwszych wierszy obiektu DataFrame do sterownika programu.
`first()` zwraca pierwszy wiersz obiektu DataFrame do sterownika programu.
`show(n)` wyświetla n pierwszych wierszy obiektu DataFrame.
`toPandas()` zwraca obiekt DataFrame jako obiekt Pandas.


### write()

`write()` zapisuje obiekt DataFrame do pliku.

In [ ]:
df = spark.read.parquet("../../data/sklep/categories")
df.write.csv("categories.csv")

## Zapytania SQL na obiektach DataFrame

Obiekty DataFrame można przekształcić na tabele tymczasowe, które można wykorzystać w zapytaniach SQL.

In [ ]:
df = spark.read.parquet("../../data/sklep/categories")
df.createOrReplaceTempView("categories")


In [ ]:
spark.sql("SELECT * FROM categories").show(5)